In [42]:
#from ipynb.fs.defs.book_search import return_liked, 
#liked_books = []
#liked_books = return_liked(liked_books)

#display(liked_books)

liked_books = ['12961964',
 '18887980',
 '18902634',
 '15769088',
 '19288043',
 '76847',
 '6457081',
 '29983711']

In [43]:
# This is user-based CF. Basically, we find a list of users who have liked similar books to the current user. Then we can make recommendations based on what other books these users liked.

book_mapping = {}
with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        book_mapping[csv_id] = book_id

In [44]:
overlap_users = set()
# overlap_users contains a list of users who liked the books
with open("goodreads_interactions.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            continue
        
        try:
            rating = int(rating)
        except ValueError:
            continue
        
        book_id = book_mapping[csv_id]
        if book_id in liked_books and rating >=4:
            overlap_users.add(user_id)

In [45]:
display(liked_books)

['12961964',
 '18887980',
 '18902634',
 '15769088',
 '19288043',
 '76847',
 '6457081',
 '29983711']

In [46]:
recs = []
# recs contains books that users who liked similar books have read.
with open("goodreads_interactions.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = book_mapping[csv_id]
            recs.append([user_id, book_id, rating])
            


In [47]:
#display(recs)

In [48]:
import pandas as pd

rec = pd.DataFrame(recs, columns = ['user_id', 'book_id', 'rating'])
rec['book_id'] = rec['book_id'].astype(str)

In [49]:
top_recs = rec['book_id'].value_counts().head(10) #counts up number of times each book occurred. shows top books
top_recs = top_recs.index.values

In [50]:
book_titles = pd.read_json("book_titles.json")
book_titles["book_id"] = book_titles["book_id"].astype(str)

In [51]:
book_titles[book_titles["book_id"].isin(top_recs)]

,book_id,title,ratings,url,cover,modified_title
255053,2767052,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
289541,41865,"Twilight (Twilight, #1)",3941381,https://www.goodreads.com/book/show/41865.Twil...,https://images.gr-assets.com/books/1361039443m...,twilight twilight 1
334328,19063,The Book Thief,1193697,https://www.goodreads.com/book/show/19063.The_...,https://images.gr-assets.com/books/1390053681m...,the book thief
415023,4671,The Great Gatsby,2758812,https://www.goodreads.com/book/show/4671.The_G...,https://images.gr-assets.com/books/1490528560m...,the great gatsby
637207,2429135,"The Girl with the Dragon Tattoo (Millennium, #1)",1858152,https://www.goodreads.com/book/show/2429135.Th...,https://images.gr-assets.com/books/1327868566m...,the girl with the dragon tattoo millennium 1
708106,2657,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird
786647,3,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
891266,19288043,Gone Girl,588451,https://www.goodreads.com/book/show/19288043-g...,https://images.gr-assets.com/books/1397056917m...,gone girl
977392,22557272,The Girl on the Train,1076144,https://www.goodreads.com/book/show/22557272-t...,https://images.gr-assets.com/books/1490903702m...,the girl on the train
980650,11870085,The Fault in Our Stars,2429317,https://www.goodreads.com/book/show/11870085-t...,https://images.gr-assets.com/books/1360206420m...,the fault in our stars


In [52]:
all_recs = rec["book_id"].value_counts()

In [53]:
all_recs = all_recs.to_frame().reset_index()

In [54]:
all_recs.columns = ["book_id", "book_count"]

In [55]:
all_recs = all_recs.merge(book_titles, how="inner", on="book_id")

In [56]:
all_recs

,book_id,book_count,title,ratings,url,cover,modified_title
0,19288043,24296,Gone Girl,588451,https://www.goodreads.com/book/show/19288043-g...,https://images.gr-assets.com/books/1397056917m...,gone girl
1,2767052,18273,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1
2,22557272,17442,The Girl on the Train,1076144,https://www.goodreads.com/book/show/22557272-t...,https://images.gr-assets.com/books/1490903702m...,the girl on the train
3,3,16843,Harry Potter and the Sorcerer's Stone (Harry P...,4765497,https://www.goodreads.com/book/show/3.Harry_Po...,https://images.gr-assets.com/books/1474154022m...,harry potter and the sorcerers stone harry pot...
4,2657,15864,To Kill a Mockingbird,3255518,https://www.goodreads.com/book/show/2657.To_Ki...,https://images.gr-assets.com/books/1361975680m...,to kill a mockingbird
...,...,...,...,...,...,...,...
793573,2411854,1,Anthem of a Reluctant Prophet,46,https://www.goodreads.com/book/show/2411854.An...,https://images.gr-assets.com/books/1320421950m...,anthem of a reluctant prophet
793574,18523217,1,"A Nightmare Come True (Nightmares, #1)",24,https://www.goodreads.com/book/show/18523217-a...,https://images.gr-assets.com/books/1380191785m...,a nightmare come true nightmares 1
793575,12120850,1,Judge Dredd: the Restricted Files 03,39,https://www.goodreads.com/book/show/12120850-j...,https://images.gr-assets.com/books/1316193454m...,judge dredd the restricted files 03
793576,7804169,1,Judge Dredd: MegaCity Masters 02,29,https://www.goodreads.com/book/show/7804169-ju...,https://s.gr-assets.com/assets/nophoto/book/11...,judge dredd megacity masters 02


In [57]:
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"] / all_recs["ratings"]) #penalizes popularity so that the recs arent always books with the most ratings
all_recs.sort_values("score", ascending=False).head(10)

,book_id,book_count,title,ratings,url,cover,modified_title,score
3813,24909347,426,"Obsidio (The Illuminae Files, #3)",82,https://www.goodreads.com/book/show/24909347-o...,https://images.gr-assets.com/books/1501704611m...,obsidio the illuminae files 3,2213.121951
6623,26856502,263,"Vengeful (Villains, #2)",35,https://www.goodreads.com/book/show/26856502-v...,https://s.gr-assets.com/assets/nophoto/book/11...,vengeful villains 2,1976.257143
3631,28170940,446,"Lethal White (Cormoran Strike, #4)",106,https://www.goodreads.com/book/show/28170940-l...,https://s.gr-assets.com/assets/nophoto/book/11...,lethal white cormoran strike 4,1876.566038
11159,36307629,163,"King of Scars (King of Scars, #1)",22,https://www.goodreads.com/book/show/36307629-k...,https://images.gr-assets.com/books/1506962795m...,king of scars king of scars 1,1207.681818
0,19288043,24296,Gone Girl,588451,https://www.goodreads.com/book/show/19288043-g...,https://images.gr-assets.com/books/1397056917m...,gone girl,1003.134698
6471,29749098,269,"Catwoman: Soulstealer (DC Icons, #3)",73,https://www.goodreads.com/book/show/29749098-c...,https://s.gr-assets.com/assets/nophoto/book/11...,catwoman soulstealer dc icons 3,991.246575
5548,30809786,309,"A Reaper at the Gates (An Ember in the Ashes, #3)",124,https://www.goodreads.com/book/show/30809786-a...,https://images.gr-assets.com/books/1507476834m...,a reaper at the gates an ember in the ashes 3,770.008065
7556,33645809,234,"Legendary (Caraval, #2)",76,https://www.goodreads.com/book/show/33645809-l...,https://images.gr-assets.com/books/1506803078m...,legendary caraval 2,720.473684
15083,31520883,122,"Untitled (An Ember in the Ashes, #4)",22,https://www.goodreads.com/book/show/31520883-u...,https://s.gr-assets.com/assets/nophoto/book/11...,untitled an ember in the ashes 4,676.545455
3078,26032825,514,"The Cruel Prince (The Folk of the Air, #1)",400,https://www.goodreads.com/book/show/26032825-t...,https://images.gr-assets.com/books/1493047153m...,the cruel prince the folk of the air 1,660.490000


In [58]:
popular_recs = all_recs[all_recs["book_count"] > 75].sort_values("score", ascending=False)

In [59]:
def clickable(val):
    return '<a target="_blank" href={}"> Goodreads </a>'.format(val)

def cover(val):
    return '<img src="{}" width=60></image>'.format(val)

best_recs = popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10)
display(best_recs.style.format({'cover':cover, 'url': clickable}))

display(best_recs)

,book_id,book_count,title,ratings,url,cover,modified_title,score
3813,24909347,426,"Obsidio (The Illuminae Files, #3)",82,Goodreads,,obsidio the illuminae files 3,2213.121951
6623,26856502,263,"Vengeful (Villains, #2)",35,Goodreads,,vengeful villains 2,1976.257143
3631,28170940,446,"Lethal White (Cormoran Strike, #4)",106,Goodreads,,lethal white cormoran strike 4,1876.566038
11159,36307629,163,"King of Scars (King of Scars, #1)",22,Goodreads,,king of scars king of scars 1,1207.681818
6471,29749098,269,"Catwoman: Soulstealer (DC Icons, #3)",73,Goodreads,,catwoman soulstealer dc icons 3,991.246575
5548,30809786,309,"A Reaper at the Gates (An Ember in the Ashes, #3)",124,Goodreads,,a reaper at the gates an ember in the ashes 3,770.008065
7556,33645809,234,"Legendary (Caraval, #2)",76,Goodreads,,legendary caraval 2,720.473684
15083,31520883,122,"Untitled (An Ember in the Ashes, #4)",22,Goodreads,,untitled an ember in the ashes 4,676.545455
3078,26032825,514,"The Cruel Prince (The Folk of the Air, #1)",400,Goodreads,,the cruel prince the folk of the air 1,660.490000
6625,29749090,263,"Batman: Nightwalker (DC Icons, #2)",119,Goodreads,,batman nightwalker dc icons 2,581.252101


,book_id,book_count,title,ratings,url,cover,modified_title,score
3813,24909347,426,"Obsidio (The Illuminae Files, #3)",82,https://www.goodreads.com/book/show/24909347-o...,https://images.gr-assets.com/books/1501704611m...,obsidio the illuminae files 3,2213.121951
6623,26856502,263,"Vengeful (Villains, #2)",35,https://www.goodreads.com/book/show/26856502-v...,https://s.gr-assets.com/assets/nophoto/book/11...,vengeful villains 2,1976.257143
3631,28170940,446,"Lethal White (Cormoran Strike, #4)",106,https://www.goodreads.com/book/show/28170940-l...,https://s.gr-assets.com/assets/nophoto/book/11...,lethal white cormoran strike 4,1876.566038
11159,36307629,163,"King of Scars (King of Scars, #1)",22,https://www.goodreads.com/book/show/36307629-k...,https://images.gr-assets.com/books/1506962795m...,king of scars king of scars 1,1207.681818
6471,29749098,269,"Catwoman: Soulstealer (DC Icons, #3)",73,https://www.goodreads.com/book/show/29749098-c...,https://s.gr-assets.com/assets/nophoto/book/11...,catwoman soulstealer dc icons 3,991.246575
5548,30809786,309,"A Reaper at the Gates (An Ember in the Ashes, #3)",124,https://www.goodreads.com/book/show/30809786-a...,https://images.gr-assets.com/books/1507476834m...,a reaper at the gates an ember in the ashes 3,770.008065
7556,33645809,234,"Legendary (Caraval, #2)",76,https://www.goodreads.com/book/show/33645809-l...,https://images.gr-assets.com/books/1506803078m...,legendary caraval 2,720.473684
15083,31520883,122,"Untitled (An Ember in the Ashes, #4)",22,https://www.goodreads.com/book/show/31520883-u...,https://s.gr-assets.com/assets/nophoto/book/11...,untitled an ember in the ashes 4,676.545455
3078,26032825,514,"The Cruel Prince (The Folk of the Air, #1)",400,https://www.goodreads.com/book/show/26032825-t...,https://images.gr-assets.com/books/1493047153m...,the cruel prince the folk of the air 1,660.490000
6625,29749090,263,"Batman: Nightwalker (DC Icons, #2)",119,https://www.goodreads.com/book/show/29749090-b...,https://images.gr-assets.com/books/1500079745m...,batman nightwalker dc icons 2,581.252101
